# 第四次作业-选做题 并发与设计模式综合

**负责助教：王瑞环**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第四次选做题+姓名+学号.ipynb, 例如 第四次选做题+张三+1000000000.ipynb</span>

<span style="color:red; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第四次选做题+张三+1000000000+#.ipynb</span>

<span style="color:red; font-weight:bold;">本次课同时发布课后练习和选做题，提交时请注意区分提交通道</span>

<span style="color:red; font-weight:bold;">由于作业中涉及线程相关知识，课上没来得及讲完。本次选做题的截止时间也会相应靠后，13号提交，有基础的同学可以先行学习并尽量9号前提交。</span>

## **第一题** 

结合多线程、工厂模式、生产者-消费者模式等，实现一个多线程工厂
- 工厂有 `WorkerA` 和 `WorkerB` 两类工人，分别可以完成任务 `TaskA` 和 `TaskB`
- 工厂每收到一个任务
  - 如果任务不属于工人能完成的任务，则输出无法处理
  - 如果任务属于工人能完成的任务，则派出一名工人，工人花费一定时间完成任务后返回工厂
  - 工厂的每种工人数目有限，如果工厂内没有对应的工人，该任务需阻塞直至有工人回来

需要补充的部分有两处
- `Factory.dispatch_task`中，向对应任务队列添加任务，注意更新该类任务的计数器
- `Factory.shutdown`中，向队列中添加停止信号，让所有工人线程能够结束，从而及时下班

In [ ]:
import threading
import queue
import random
import time


class Task:
    def __init__(self, task_id, task_type):
        self.task_id = task_id
        self.task_type = task_type

    def __str__(self):
        return f"任务{self.task_type}-{self.task_id}"


class Worker(threading.Thread):
    def __init__(self, worker_type, worker_id, task_queue):
        super().__init__()
        self.worker_type = worker_type
        self.worker_id = worker_id
        self.task_queue = task_queue
    
    def __str__(self):
        return f"工人{self.worker_type}-{self.worker_id}"
    
    def run(self):
        while True:
            task = self.task_queue.get()
            if task is None:  # 停止信号
                self.task_queue.task_done()
                break
            
            print(f"{self} 开始执行 {task}")
            t = random.random() * 4 + 1
            time.sleep(t) 
            print(f"{self} 完成 {task}，用时 {t:.2f} 秒")
            
            self.task_queue.task_done()


class Factory:
    def __init__(self, max_workers_A, max_workers_B):
        self.task_queue_A = queue.Queue()
        self.task_queue_B = queue.Queue()

        self.workers_A = []
        self.workers_B = []

        # 初始化A类工人并启动
        for i in range(1, max_workers_A + 1):
            worker = Worker("A", i, self.task_queue_A)
            worker.start()
            self.workers_A.append(worker)

        # 初始化B类工人并启动
        for i in range(1, max_workers_B + 1):
            worker = Worker("B", i, self.task_queue_B)
            worker.start()
            self.workers_B.append(worker)

        self.task_counter_A = 0 # 任务A计数器
        self.task_counter_B = 0 # 任务B计数器

    def dispatch_task(self, task_type):
        print(f"工厂接收到任务: {task_type}")
        
        if task_type == 'A':
            self.task_counter_A += 1
            task = Task(self.task_counter_A, 'A')
            self.task_queue_A.put(task)
            print(f"工厂分配 {task} 到A类队列")
        elif task_type == 'B':
            self.task_counter_B += 1
            task = Task(self.task_counter_B, 'B')
            self.task_queue_B.put(task)
            print(f"工厂分配 {task} 到B类队列")
        else:
            print(f"工厂无法处理未知任务类型: {task_type}")

    def shutdown(self):
        # 向每个队列添加停止信号，让所有工人线程退出
        for _ in self.workers_A:
            self.task_queue_A.put(None)
        for _ in self.workers_B:
            self.task_queue_B.put(None)
        
        # 等待所有任务完成
        self.task_queue_A.join()
        self.task_queue_B.join()
        
        # 等待所有线程结束
        for worker in self.workers_A + self.workers_B:
            worker.join()
        
        print("所有任务处理完成，下班！")


factory = Factory(max_workers_A=3, max_workers_B=2)

tasks_types = ['A'] * 5 + ['B'] * 5 + ['C', 'D', 'E']
random.shuffle(tasks_types)
for task_type in tasks_types:
    time.sleep(random.random())
    factory.dispatch_task(task_type)
    
factory.shutdown()

你应当能看到与下方例子类似的输出
```
工厂接收到任务: A
工厂分配 任务A-1 到A类队列
工人A-1 开始执行 任务A-1
工厂接收到任务: B
工厂分配 任务B-1 到B类队列
工人B-1 开始执行 任务B-1
工厂接收到任务: A
工厂分配 任务A-2 到A类队列
工人A-2 开始执行 任务A-2
工厂接收到任务: B
工厂分配 任务B-2 到B类队列
工人B-2 开始执行 任务B-2
工人B-1 完成 任务B-1，用时 1.75 秒
工厂接收到任务: A
工厂分配 任务A-3 到A类队列
工人A-3 开始执行 任务A-3
工厂接收到任务: D
工厂无法处理未知任务类型: D
工人A-1 完成 任务A-1，用时 2.59 秒
工厂接收到任务: E
工厂无法处理未知任务类型: E
工厂接收到任务: B
工厂分配 任务B-3 到B类队列
工人B-1 开始执行 任务B-3
工厂接收到任务: C
工厂无法处理未知任务类型: C
工人A-2 完成 任务A-2，用时 4.12 秒
工厂接收到任务: B
工厂分配 任务B-4 到B类队列
工厂接收到任务: B
工厂分配 任务B-5 到B类队列
工厂接收到任务: A
工厂分配 任务A-4 到A类队列
工人A-1 开始执行 任务A-4
工人B-2 完成 任务B-2，用时 4.43 秒
工人B-2 开始执行 任务B-4
工厂接收到任务: A
工厂分配 任务A-5 到A类队列
工人A-2 开始执行 任务A-5
工人A-3 完成 任务A-3，用时 3.87 秒
工人A-1 完成 任务A-4，用时 1.95 秒
工人A-2 完成 任务A-5，用时 1.71 秒
工人B-1 完成 任务B-3，用时 4.94 秒
工人B-1 开始执行 任务B-5
工人B-2 完成 任务B-4，用时 4.62 秒
工人B-1 完成 任务B-5，用时 4.83 秒
所有任务处理完成，下班！
```

## **第二题**

在课后练习1.4中，我们已经实现了一个非常简易的教学系统，在本次题目中，我们将同样以发布-订阅模式为主线，为其拓展更多功能

In [1]:
import threading
import time
import random

## **Step 1.** 通用的发布-订阅模式事件管理器

完成`EventManager`类，至少包括
- `EventManager.subscribe`方法：用于回调函数`callback`对事件`event_type`的订阅
- `EventManager.unsubscribe`方法：用于取消回调函数`callback`对事件`event_type`的订阅
- `EventManager.publish`方法：用于发布类型为`event_type`，并提供对应的数据`data`作为订阅了`event_type`的回调函数的参数；（可选）进行异常处理
- `EventManager.clear`方法：清除所有订阅

研究后面的测试样例将帮助你进一步理解各个类方法的作用

In [ ]:
class EventManager:
    _subscribers = {}
    _lock = threading.Lock()

    @classmethod
    def subscribe(cls, event_type, callback):
        with cls._lock:
            if event_type not in cls._subscribers:
                cls._subscribers[event_type] = []
            cls._subscribers[event_type].append(callback)

    @classmethod
    def unsubscribe(cls, event_type, callback):
        with cls._lock:
            if event_type in cls._subscribers:
                try:
                    cls._subscribers[event_type].remove(callback)
                except ValueError:
                    pass

    @classmethod
    def publish(cls, event_type, data):
        # Copy callbacks while holding the lock to avoid race conditions
        with cls._lock:
            callbacks = cls._subscribers.get(event_type, []).copy()
        for callback in callbacks:
            try:
                callback(data)
            except Exception as e:
                pass

    @classmethod
    def clear(cls):
        with cls._lock:
            cls._subscribers.clear()

以下是基础功能测试，可以帮助你理解各个方法的作用

In [ ]:
def test_eventmanager_basic():

    EventManager.clear()
    
    def listener1(data):
        print(f"Listener1 received: {data}, add by 1 to get {data + 1}")

    def listener2(data):
        print(f"Listener2 received: {data}, add by 2 to get {data + 2}")

    # 函数 listener1 和 listener2 订阅了事件 test_event1
    EventManager.subscribe("test_event1", listener1)
    EventManager.subscribe("test_event1", listener2)
    # 函数 listener2 订阅了事件 test_event2
    EventManager.subscribe("test_event2", listener2)

    # 发布事件 test_event1，带有数据 1
    # 由于 listener1 和 listener2 都订阅了 test_event1，所以它们都会被调用
    # 因此，我们预期会看到两行输出，分别是 listener1 和 listener2 关于 1 的输出
    print("Publishing 'test_event1' with data 1")
    EventManager.publish("test_event1", 1)

    # 发布事件 test_event2，带有数据 2
    # 由于只有 listener2 订阅了 test_event2，所以只会看到 listener2 的输出
    print("Publishing 'test_event2' with data 2")
    EventManager.publish("test_event2", 2)

    # 取消 listener2 对 test_event1 的订阅
    # 并再次发布 test_event1，带有数据 3
    # 这次只会看到 listener1 关于 3 的输出
    EventManager.unsubscribe("test_event1", listener2)
    print("Publishing 'test_event1' again with data 3")
    EventManager.publish("test_event1", 3)
    
    # 如果你实现了异常处理功能，那么下面的代码应该不会导致程序崩溃
    # EventManager.publish("test_event1", "bad data")
    # print("Your event manager is robust!")
    
    # 清除所有订阅
    EventManager.clear()

test_eventmanager_basic()

Publishing 'test_event1' with data 1
Listener1 received: 1, add by 1 to get 2
Listener2 received: 1, add by 2 to get 3
Publishing 'test_event2' with data 2
Listener2 received: 2, add by 2 to get 4
Publishing 'test_event1' again with data 3
Listener1 received: 3, add by 1 to get 4


以下是多线程测试，如果你的`EventManager`不支持多线程或者无法确保线程安全，可以跳过

后续的测试均为单线程的情况，因此是否通过以下测试不影响后面内容的完成

In [ ]:
def event_manager_multithreading():
    EventManager.clear()
    
    lock = threading.Lock()
    
    results = []
    def callback(data):
        with lock:
            results.append(data)
    
    EventManager.subscribe("multithreading", callback)
    
    threads = [
        threading.Thread(
            target=lambda: EventManager.publish("multithreading", i)
        ) for i in range(1000)
    ]
    
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    
    assert len(results) == 1000
    
    EventManager.clear()
    print("Multithreading test passed!")

event_manager_multithreading()

## **Step 2.** 发布-订阅事件分析设计

为了实现学生教师发布通知/作业、学生提交作业、作业打分等功能，我们至少应该需要教师、学生、课程等类，每一类的不同类方法应当订阅不同的事件

- 通知与作业的发布：由教师类发布，学生和课程类订阅
- 作业提交：由学生类发布，教师和课程类订阅
- 作业批改：由教师类发布，学生和课程类订阅
  
由此可以分析出各个类所应该实现的方法
- 教师类
  - 发布通知（发布）
  - 发布作业（发布）
  - 接受作业（订阅）
  - 发布分数（发布）
- 学生类
  - 接受通知（订阅）
  - 接受作业（订阅）
  - 提交作业（发布）
  - 接受成绩（订阅）
- 课程类
  - 存储教师发布的通知（订阅）
  - 存储教师发布的作业（订阅）
  - 存储学生提交的作业（订阅）
  - 存储教师批改作业成绩（订阅）

此外，应当需要一个选课管理系统，实现设置任课教师、学生选课退课功能

## **Step 3.** 教师、学生、课程类的实现

教师类`Teacher`的一种写法已给出，你可以参考其格式和方法，结合 Step 2 中的分析，按照程序中的要求完成学生类`Student`和课程类`Course`

你可以对代码进行任意的修改，包括但不限于定义新函数、修改各个类方法的参数表、设置或修改各个类的成员变量、设置新的类、修改或重构教师类等

In [ ]:
class Teacher:
    def __init__(self, uid, name):
        self.uid = uid
        self.name = name
        self.courses = []       # 该老师教授的课程
    
    def __str__(self):
        return f"教师 {self.name}"
    
    # 发布通知
    def announce_notice(self, course, notice, content):
        print(f'{self} 在 {course} 中发布了 {notice}')
        
        data = {
            "teacher": self,        # 发布通知的老师
            "course": course,       # 发布通知的课程
            "notice": notice,       # 通知标题
            "content": content      # 通知内容
        }
        
        # 将通知信息发布到所有订阅了 notice_announcement:{course.uid} 的订阅者
        # 因此，(1) 在实现学生和课程类时，其接收通知的方法所处理的输入格式应该与这里的 data 一致
        # (2) 在后续建立选课系统的过程中，这门课程和选了这门课的学生都应该订阅 notice_announcement:{course.uid} 以收到这条通知
        EventManager.publish(f"notice_announcement:{course.uid}", data)
    
    # 发布作业
    def announce_homework(self, course, homework, content):
        print(f'{self} 在 {course} 中发布了 {homework}')
        
        data = {
            "teacher": self,        # 发布作业的老师
            "course": course,       # 发布作业的课程
            "homework": homework,   # 作业标题
            "content": content      # 作业内容
        }
        
        EventManager.publish(f"homework_announcement:{course.uid}", data)
        
    # 发布成绩
    def announce_grade(self, course, homework, student, grade):
        print(f'{self} 批改了 {student} 在 {course} 中的 {homework}')
        
        data = {
            "teacher": self,        # 批改作业的老师
            "course": course,       # 作业所属课程
            "student": student,     # 作业提交者
            "homework": homework,   # 作业标题
            "grade": grade          # 作业成绩
        }
        
        EventManager.publish(f"grading_announcement:{course.uid}:{student.uid}", data)
    
    # 接受作业
    def receive_homework(self, data):
        course = data["course"]         # 作业所属课程
        student = data["student"]       # 作业提交者
        homework = data["homework"]     # 作业标题
        
        print(f'{self} 收到了 {student} 在 {course} 中提交的 {homework}')
               
        # 模拟批改作业用时
        time.sleep(random.random() * 2)
        # 模拟吹风机
        grade = random.randint(60, 100)
        
        # 发布作业成绩
        self.announce_grade(course, homework, student, grade)

In [ ]:
class Student:
    def __init__(self, uid, name):
        self.uid = uid
        self.name = name
        self.courses = []       # 该学生选修的课程
    
    def __str__(self):
        return f"学生 {self.name}"

    def receive_notice(self, data):
        print(f'{self} 收到了 {data["teacher"]} 在 {data["course"]} 中发布的 {data["notice"]}')
    
    def receive_homework(self, data):       
        print(f'{self} 收到了 {data["teacher"]} 在 {data["course"]} 中发布的 {data["homework"]}')
        time.sleep(random.random() * 2)
        # 调用 submit_homework 方法提交作业
        self.submit_homework(data["course"], data["homework"], f'{data["homework"]} 的解答')
    
    def receive_grade(self, data):
        print(f'{self} 收到了 {data["teacher"]} 在 {data["course"]} 中发布的 {data["homework"]} 的成绩：{data["grade"]}')
    
    def submit_homework(self, course, homework, submission):
        print(f'{self} 提交了 {course} 中的 {homework}')
        data = {
            "student": self,
            "course": course,
            "homework": homework,
            "submission": submission
        }
        EventManager.publish(f"homework_submission:{course.uid}", data)
        

In [ ]:
class Course:
    def __init__(self, uid, name):
        self.uid = uid
        self.name = name
        self.teacher = None
        self.students = []
        self.notices = []
        self.homeworks_announced = []
        self.homework_submissions = []
        self.grades = []
    
    def __str__(self):
        return f"课程 {self.name}"
    
    def receive_notice(self, data):
        self.notices.append(data)
    
    def receive_homework_announcement(self, data):
        self.homeworks_announced.append(data)

    def receive_grade(self, data):
        self.grades.append(data)
    
    def receive_homework_submission(self, data):
        if data["student"] in self.students:
            self.homework_submissions.append(data)

## **Step 4.** 选课管理函数

在这一步中，需要对教师开设课程、学生选课、学生退课等情况，设置订阅内容，请完成
- `offer_course`模拟教师开课
  - 更新教师的开设课程信息
  - 更新课程的授课教师信息
  - 课程需要订阅教师发布的同志、作业
- `select_course`模拟学生选课
  - 更新学生的选修课程信息
  - 更新课程的选修学生信息
  - 学生需要订阅教师发布的通知、作业、成绩
  - 课程需要订阅学生提交的作业、教师发布的成绩
  - 教师需要订阅学生提交的作业
- `quit_course`模拟学生退课
  - 更新学生的选修课程信息
  - 更新课程的选修学生信息
  - 取消所有在`select_course`中的订阅

In [ ]:
def offer_course(teacher, course):
    if course not in teacher.courses:
        teacher.courses.append(course)
    course.teacher = teacher
    EventManager.subscribe(f"notice_announcement:{course.uid}", course.receive_notice)
    EventManager.subscribe(f"homework_announcement:{course.uid}", course.receive_homework_announcement)
    EventManager.subscribe(f"homework_submission:{course.uid}", teacher.receive_homework)

def select_course(student, course):
    if course not in student.courses:
        student.courses.append(course)
    if student not in course.students:
        course.students.append(student)

    EventManager.subscribe(f"notice_announcement:{course.uid}", student.receive_notice)
    EventManager.subscribe(f"homework_announcement:{course.uid}", student.receive_homework)
    EventManager.subscribe(f"grading_announcement:{course.uid}:{student.uid}", student.receive_grade)
    EventManager.subscribe(f"homework_submission:{course.uid}", course.receive_homework_submission)
    EventManager.subscribe(f"grading_announcement:{course.uid}:{student.uid}", course.receive_grade)

def quit_course(student, course):
    if course in student.courses:
        student.courses.remove(course)
    if student in course.students:
        course.students.remove(student)
    EventManager.unsubscribe(f"notice_announcement:{course.uid}", student.receive_notice)
    EventManager.unsubscribe(f"homework_announcement:{course.uid}", student.receive_homework)
    EventManager.unsubscribe(f"grading_announcement:{course.uid}:{student.uid}", student.receive_grade)
    EventManager.unsubscribe(f"grading_announcement:{course.uid}:{student.uid}", course.receive_grade)

## **Step 5.** 基本功能测试

通过以下代码验证你所实现的教师、学生、课程类和选课管理函数是否符合预期

In [9]:
EventManager.clear()

# 创建教师、学生和课程
teacher = Teacher('001', '胡老师')
student1 = Student('001', '小明')
student2 = Student('002', '小红')
course = Course('001', 'Python 程序设计与数据科学导论')

# 教师开设课程
offer_course(teacher, course)

# 学生选课
select_course(student1, course)
select_course(student2, course)

In [10]:
# 模拟教师发布通知
teacher.announce_notice(course, '第一次课程通知', '本周的课程取消')

教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布了 第一次课程通知
学生 小明 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程通知
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程通知


In [11]:
# 模拟教师发布作业
teacher.announce_homework(course, '第一次课程作业', 'Python数据类型')

教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布了 第一次课程作业
学生 小明 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程作业
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程作业


学生 小明 提交了 课程 Python 程序设计与数据科学导论 中的 第一次课程作业
教师 胡老师 收到了 学生 小明 在 课程 Python 程序设计与数据科学导论 中提交的 第一次课程作业
学生 小红 提交了 课程 Python 程序设计与数据科学导论 中的 第一次课程作业
教师 胡老师 收到了 学生 小红 在 课程 Python 程序设计与数据科学导论 中提交的 第一次课程作业
教师 胡老师 批改了 学生 小明 在 课程 Python 程序设计与数据科学导论 中的 第一次课程作业
学生 小明 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程作业 的成绩
教师 胡老师 批改了 学生 小红 在 课程 Python 程序设计与数据科学导论 中的 第一次课程作业
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第一次课程作业 的成绩


In [ ]:
print("\n------------------------------")
print("作业：第一次课程作业")
first_hw = next(hw for hw in course.homeworks_announced if hw['homework'] == '第一次课程作业')
print(f"作业内容：{first_hw['content']}")
print()
for student in [student1, student2]:
    submission = next((sub for sub in course.homework_submissions if sub['student'] == student and sub['homework'] == '第一次课程作业'), None)
    grade = next((g for g in course.grades if g['student'] == student and g['homework'] == '第一次课程作业'), None)
    if submission and grade:
        print(f"学生：{student.name}")
        print(f"提交内容：{submission['submission']}")
        print(f"成绩：{grade['grade']}\n")

------------------------------
作业：第一次课程作业
作业内容：Python数据类型

学生：小明
提交内容：第一次课程作业 的解答
成绩：66

学生：小红
提交内容：第一次课程作业 的解答
成绩：73


In [13]:
# 学生退课后，教师再次发布通知
quit_course(student1, course)
teacher.announce_notice(course, '第二次课程通知', '本周的课程加时')

教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布了 第二次课程通知
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第二次课程通知


In [ ]:
# 学生退课后，教师再次发布作业
quit_course(student1, course)
teacher.announce_homework(course, '第二次课程作业', 'Python函数')

教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布了 第二次课程作业
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第二次课程作业


学生 小红 提交了 课程 Python 程序设计与数据科学导论 中的 第二次课程作业
教师 胡老师 收到了 学生 小红 在 课程 Python 程序设计与数据科学导论 中提交的 第二次课程作业
教师 胡老师 批改了 学生 小红 在 课程 Python 程序设计与数据科学导论 中的 第二次课程作业
学生 小红 收到了 教师 胡老师 在 课程 Python 程序设计与数据科学导论 中发布的 第二次课程作业 的成绩


In [ ]:
print("------------------------------")
print("作业：第二次课程作业")
second_hw = next(hw for hw in course.homeworks_announced if hw['homework'] == '第二次课程作业')
print(f"作业内容：{second_hw['content']}")
print()
for student in course.students:
    submission = next((sub for sub in course.homework_submissions if sub['student'] == student and sub['homework'] == '第二次课程作业'), None)
    grade = next((g for g in course.grades if g['student'] == student and g['homework'] == '第二次课程作业'), None)
    if submission and grade:
        print(f"学生：{student.name}")
        print(f"提交内容：{submission['submission']}")
        print(f"成绩：{grade['grade']}\n")

------------------------------
作业：第一次课程作业
作业内容：Python数据类型

学生：小明
提交内容：第一次课程作业 的解答
成绩：66

学生：小红
提交内容：第一次课程作业 的解答
成绩：73
------------------------------
作业：第二次课程作业
作业内容：Python函数

学生：小红
提交内容：第二次课程作业 的解答
成绩：95
